<a href="https://colab.research.google.com/github/guilhermedossantos/TCC_NN_SARIMA/blob/master/teste2_regularizando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Primeiro teste de uma rede RNN em um SARIMA 

 - Foi gerado no R um processo $SARIMA(1,0,0)\times(1,0,0)$

In [0]:
#### SETUP ####
# Keras
from keras.models import Sequential # modelo
from keras.layers import Dense, SimpleRNN, LSTM, Reshape # camadas
from keras.callbacks import EarlyStopping # early stopping

# scaler
from sklearn.preprocessing import MinMaxScaler

# Importação do arquivo 
from google.colab import files # upload de arquivos da máquina
import pandas_datareader as web # pacote bem interessante que pode valer a pena explorar (para importação de dados de diversar fontes)

# Gerais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats # usado no qq-plot
import plotly.tools as tls # para tentativa de converter mpl to plotly

# Removendo warnings
import warnings
warnings.filterwarnings('ignore') # tem também a opção 'once' para mostrar apenas uma vez

In [27]:
serie = pd.read_csv("https://raw.githubusercontent.com/guilhermedossantos/TCC_NN_SARIMA/master/serie_sarima.csv")
serie

,x
0,-0.281740
1,-2.822337
2,-0.385268
3,0.270453
4,-0.365296
...,...
1195,-1.075860
1196,0.434046
1197,-1.894967
1198,-1.868699


In [28]:
serie_original = serie

scaler = MinMaxScaler(feature_range=(0,1))
serie = scaler.fit_transform(serie_original)

x = np.array(serie[0:1199])
y = np.array(serie[1:])
x.shape
y.shape

(1199, 1)

In [29]:
treinox = x[0:1000].reshape((1000,1,1))
treinoy = y[0:1000].reshape((1000,1))
testex = x[1000:].reshape((-1,1,1))
testey = y[1000:].reshape((-1,1,))

print(testex.shape)
print(testey.shape)

(199, 1, 1)
(199, 1)


In [0]:
model = Sequential()
model.add(SimpleRNN(128, activation='relu', input_shape = (1,1)))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer = 'adam', metrics=['mape', 'mae', 'mse'])

In [31]:
model.fit(treinox, treinoy, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 251us/step - loss: 0.2002 - mape: 140142.1250 - mae: 0.4040 - mse: 0.2002
Epoch 2/5
1000/1000 [==============================] - 0s 77us/step - loss: 0.0256 - mape: 226094.4688 - mae: 0.1256 - mse: 0.0256
Epoch 3/5
1000/1000 [==============================] - 0s 55us/step - loss: 0.0103 - mape: 277503.5938 - mae: 0.0814 - mse: 0.0103
Epoch 4/5
1000/1000 [==============================] - 0s 60us/step - loss: 0.0096 - mape: 262359.9375 - mae: 0.0787 - mse: 0.0096
Epoch 5/5
1000/1000 [==============================] - 0s 64us/step - loss: 0.0090 - mape: 259381.0156 - mae: 0.0764 - mse: 0.0090


In [32]:
model.evaluate(testex,testey)

199/199 [==============================] - 0s 247us/step


[0.008340680391420071,
 10.768054008483887,
 0.07250042259693146,
 0.008340680040419102]

In [0]:
preds = model.predict(testex)
preds_original = scaler.inverse_transform(preds)

y_original = scaler.inverse_transform(testey)

mape = np.mean(abs((y_original - preds_original)/y_original))
mae = np.mean(abs(y_original - preds_original))
mse = np.mean((y_original - preds_original)**2)


print(mape)
print(mse)
print(mae)